In [1]:
import pandas as pd
import numpy as np
import json 

In [9]:
style_fake_data = pd.read_json("/home/zgy/wudongqi/GET/GET/data/gossipcop_v3-2_content_based_fake.json",orient='index')
style_fake_data

,origin_id,origin_label,origin_text,generated_text,generated_tone,generated_label,has_top_img
gossipcop-2493749932,gossipcop-2493749932,fake,Congratulations might be in order for Miley Cy...,"According to recent reports, Miley Cyrus and L...",an objective and professional tone,fake,1
gossipcop-4580247171,gossipcop-4580247171,fake,Paris Jackson and Cara Delevingne were spotted...,Paris Jackson and Cara Delevingne were seen to...,a neutral tone,fake,0
gossipcop-941805037,gossipcop-941805037,fake,Thousands are taking the streets to protest Pr...,"On Tax Day, thousands of people took to the st...",a neutral tone,fake,1
gossipcop-2547891536,gossipcop-2547891536,fake,We'd venture to say that Cindy Crawford's daug...,"Kaia Gerber, the 14-year-old daughter of super...",an objective and professional tone,fake,1
gossipcop-5189580095,gossipcop-5189580095,fake,"During the summer of 1995, John F. Kennedy Jr....","In the summer of 1995, John F. Kennedy Jr. sec...",a neutral tone,fake,1
...,...,...,...,...,...,...,...
gossipcop-907293,gossipcop-907293,legitimate,Gigi & Zayn Gave Us the Best Idea For a Valent...,"""OMG, Gigi & Zayn's Valentine's Day Gift Idea ...",a sensational tone,real,1
gossipcop-891749,gossipcop-891749,legitimate,2017 Hollywood Film Awards ceremony took place...,"""The 2017 Hollywood Film Awards ceremony, the ...",a sensational tone,real,1
gossipcop-941486,gossipcop-941486,legitimate,Jada Pinkett Smith recalled that heartbreaking...,EXCLUSIVE: Jada Pinkett Smith reveals heart-wr...,a sensational tone,real,1
gossipcop-953143,gossipcop-953143,legitimate,Another one of Luann de Lesseps' Real Housewiv...,"""SHOCKER! Real Housewives Star Luann de Lessep...",a sensational tone,real,1


In [11]:
style_fake_data[style_fake_data.has_top_img==1].count()[0]

12332

In [29]:
# 1. 过滤掉没有图片的数据
filter_data = style_fake_data[style_fake_data.has_top_img==1]
# 2. 只保留文本ID  图片ID 修改后的文本内容 图片 标签
keep_data = filter_data[["origin_id","generated_text","generated_label"]]
keep_data


,origin_id,generated_text,generated_label
gossipcop-2493749932,gossipcop-2493749932,"According to recent reports, Miley Cyrus and L...",fake
gossipcop-941805037,gossipcop-941805037,"On Tax Day, thousands of people took to the st...",fake
gossipcop-2547891536,gossipcop-2547891536,"Kaia Gerber, the 14-year-old daughter of super...",fake
gossipcop-5189580095,gossipcop-5189580095,"In the summer of 1995, John F. Kennedy Jr. sec...",fake
gossipcop-9588339534,gossipcop-9588339534,"In 2016, several celebrities faced major contr...",fake
...,...,...,...
gossipcop-907293,gossipcop-907293,"""OMG, Gigi & Zayn's Valentine's Day Gift Idea ...",real
gossipcop-891749,gossipcop-891749,"""The 2017 Hollywood Film Awards ceremony, the ...",real
gossipcop-941486,gossipcop-941486,EXCLUSIVE: Jada Pinkett Smith reveals heart-wr...,real
gossipcop-953143,gossipcop-953143,"""SHOCKER! Real Housewives Star Luann de Lessep...",real


In [75]:
# 3. 保存到文件中，文本和图像一一对应。
# 划分训练和测试数据集，要求训练集和测试集中的正负类的分布差不多，
# 其中 文本 ID 和图像ID一致，也对应上文本的存储位置。
df_shuffled = keep_data.sample(frac=1.0, random_state=42)

# 定义训练集和测试集的比例
train_ratio = 0.8  # 训练集比例
test_ratio = 1 - train_ratio  # 测试集比例

# 计算训练集和测试集的数量
num_train_samples = int(len(df_shuffled) * train_ratio)
num_test_samples = len(df_shuffled) - num_train_samples

# 分割数据集
train_set = df_shuffled.iloc[:num_train_samples]
test_set = df_shuffled.iloc[num_train_samples:]



In [76]:
train_set.groupby("generated_label").count()

,origin_id,generated_text
generated_label,,
fake,2198,2198
real,7667,7667


In [77]:
test_set.groupby("generated_label").count()

,origin_id,generated_text
generated_label,,
fake,518,518
real,1949,1949


In [78]:
train_set["generated_text"] = train_set["generated_text"].map(lambda x: str(x)[:500])

/usr/local/miniconda3/envs/yj_c2dsr/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [88]:
test_set["generated_text"] = test_set["generated_text"].map(lambda x: str(x)[:500])

/usr/local/miniconda3/envs/yj_c2dsr/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [86]:
train_set.to_json("/home/zgy/yanjie/gossipcop_dataset/gossipcop_v3-1_style_based_fake_train.json", orient="records", indent=4,)

In [89]:
test_set.to_json("/home/zgy/yanjie/gossipcop_dataset/gossipcop_v3-1_style_based_fake_test.json", orient="records", indent=4,)

In [4]:

df_train = pd.read_json("/home/zgy/yanjie/gossipcop_dataset/gossipcop_v3-1_style_based_fake_train.json")

In [6]:
tweet_list = []
tweet_list = df_train["generated_text"].tolist()
tweet_list

["The heartwarming moment a 15-month-old baby stole the show during an interview with Jonathan Rhys Meyers today, as he crashed the set to say hi to his beloved father.\n\nLittle Wolf, Jonathan Rhys Meyers' adorable baby son, was backstage with his mother, Mara Lane, while the Irish actor appeared on Good Morning Britain to promote his new film, Damascus Cover.\n\nAs cameras quickly cut to the toddler, he couldn't resist the urge to make his way to the set and inspect what his father was up to. Jonath",
 "Mediaite reported that In Touch magazine's recent cover story claiming that Megyn Kelly would replace Matt Lauer on NBC's Today show was not based on credible sources. Gossip Cop, a sister site of Mediaite, debunked the story as being entirely fabricated. The tabloid offered absurd evidence, such as claiming Kelly got a joint interview with President Trump and Russian President Vladimir Putin, which is unlikely to happen. Gossip Cop also noted that Lauer will continue to be the face o

In [7]:
label_dict = {'fake': [0, 1], 'real': [1, 0]}

In [11]:
df_train["generated_label"].map(lambda x: label_dict[x]).tolist()

[[1, 0],
 [0, 1],
 [1, 0],
 [1, 0],
 [1, 0],
 [0, 1],
 [1, 0],
 [0, 1],
 [1, 0],
 [1, 0],
 [1, 0],
 [1, 0],
 [1, 0],
 [1, 0],
 [0, 1],
 [1, 0],
 [1, 0],
 [1, 0],
 [1, 0],
 [1, 0],
 [1, 0],
 [1, 0],
 [1, 0],
 [0, 1],
 [1, 0],
 [1, 0],
 [1, 0],
 [0, 1],
 [1, 0],
 [1, 0],
 [1, 0],
 [1, 0],
 [1, 0],
 [0, 1],
 [0, 1],
 [1, 0],
 [0, 1],
 [1, 0],
 [0, 1],
 [0, 1],
 [1, 0],
 [1, 0],
 [1, 0],
 [0, 1],
 [0, 1],
 [1, 0],
 [1, 0],
 [1, 0],
 [0, 1],
 [1, 0],
 [1, 0],
 [1, 0],
 [0, 1],
 [0, 1],
 [0, 1],
 [1, 0],
 [1, 0],
 [1, 0],
 [1, 0],
 [1, 0],
 [1, 0],
 [1, 0],
 [1, 0],
 [0, 1],
 [1, 0],
 [1, 0],
 [1, 0],
 [1, 0],
 [1, 0],
 [0, 1],
 [1, 0],
 [1, 0],
 [1, 0],
 [0, 1],
 [1, 0],
 [1, 0],
 [1, 0],
 [1, 0],
 [1, 0],
 [1, 0],
 [1, 0],
 [1, 0],
 [1, 0],
 [1, 0],
 [1, 0],
 [1, 0],
 [1, 0],
 [1, 0],
 [1, 0],
 [0, 1],
 [1, 0],
 [1, 0],
 [1, 0],
 [1, 0],
 [1, 0],
 [1, 0],
 [1, 0],
 [0, 1],
 [1, 0],
 [1, 0],
 [1, 0],
 [1, 0],
 [1, 0],
 [0, 1],
 [1, 0],
 [1, 0],
 [1, 0],
 [0, 1],
 [1, 0],
 [0, 1],
 [1, 0],
 

In [14]:
corpus_dir = '../gossipcop_dataset'
img_dir = "top_img"
import os 
df_train["origin_id"].map(lambda x: corpus_dir+"/"+img_dir+"/" + str(x)+".png")

0       ../gossipcop_dataset/top_img/gossipcop-917567.png
1       ../gossipcop_dataset/top_img/gossipcop-4575106...
2       ../gossipcop_dataset/top_img/gossipcop-852580.png
3       ../gossipcop_dataset/top_img/gossipcop-842391.png
4       ../gossipcop_dataset/top_img/gossipcop-884638.png
                              ...                        
9860    ../gossipcop_dataset/top_img/gossipcop-897503.png
9861    ../gossipcop_dataset/top_img/gossipcop-887194.png
9862    ../gossipcop_dataset/top_img/gossipcop-916489.png
9863    ../gossipcop_dataset/top_img/gossipcop-8862742...
9864    ../gossipcop_dataset/top_img/gossipcop-902143.png
Name: origin_id, Length: 9865, dtype: object

In [21]:
from PIL import Image
from torchvision import transforms
def picture_filter(im_path):
    im = Image.open(im_path, 'r').convert('RGB')
    trans = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ])
    im = trans(im)  # type is tensor

    return im

In [27]:
img_list_path = df_train["origin_id"].map(lambda x: img_dir+"/" + str(x)+"_top_img.png").tolist()

In [29]:
img_list = []
for img in img_list_path:
    img_list.append(picture_filter(img))
    print(img)

top_img/gossipcop-917567_top_img.png
top_img/gossipcop-4575106109_top_img.png
top_img/gossipcop-852580_top_img.png
top_img/gossipcop-842391_top_img.png
top_img/gossipcop-884638_top_img.png
top_img/gossipcop-3693445154_top_img.png


KeyboardInterrupt: 

In [30]:
data_spilt = np.linspace(0, 9865, 16+1,)

In [38]:
img_list_path


['top_img/gossipcop-917567_top_img.png',
 'top_img/gossipcop-4575106109_top_img.png',
 'top_img/gossipcop-852580_top_img.png',
 'top_img/gossipcop-842391_top_img.png',
 'top_img/gossipcop-884638_top_img.png',
 'top_img/gossipcop-3693445154_top_img.png',
 'top_img/gossipcop-914978_top_img.png',
 'top_img/gossipcop-4894634035_top_img.png',
 'top_img/gossipcop-881670_top_img.png',
 'top_img/gossipcop-955383_top_img.png',
 'top_img/gossipcop-885589_top_img.png',
 'top_img/gossipcop-886957_top_img.png',
 'top_img/gossipcop-898352_top_img.png',
 'top_img/gossipcop-918778_top_img.png',
 'top_img/gossipcop-5695834478_top_img.png',
 'top_img/gossipcop-848738_top_img.png',
 'top_img/gossipcop-917881_top_img.png',
 'top_img/gossipcop-914926_top_img.png',
 'top_img/gossipcop-937078_top_img.png',
 'top_img/gossipcop-870640_top_img.png',
 'top_img/gossipcop-880641_top_img.png',
 'top_img/gossipcop-902431_top_img.png',
 'top_img/gossipcop-848816_top_img.png',
 'top_img/gossipcop-44021597_top_img.png'

In [43]:
splits = np.array_split(img_list_path, 3)

In [44]:
for sub in splits:
    print(sub)

['top_img/gossipcop-917567_top_img.png'
 'top_img/gossipcop-4575106109_top_img.png'
 'top_img/gossipcop-852580_top_img.png' ...
 'top_img/gossipcop-947687_top_img.png'
 'top_img/gossipcop-915071_top_img.png'
 'top_img/gossipcop-944130_top_img.png']
['top_img/gossipcop-888362_top_img.png'
 'top_img/gossipcop-894910_top_img.png'
 'top_img/gossipcop-866888_top_img.png' ...
 'top_img/gossipcop-6147600959_top_img.png'
 'top_img/gossipcop-896198_top_img.png'
 'top_img/gossipcop-4678149709_top_img.png']
['top_img/gossipcop-1169823957_top_img.png'
 'top_img/gossipcop-862526_top_img.png'
 'top_img/gossipcop-4517095073_top_img.png' ...
 'top_img/gossipcop-916489_top_img.png'
 'top_img/gossipcop-8862742609_top_img.png'
 'top_img/gossipcop-902143_top_img.png']


In [47]:
test_set = pd.read_json("/home/zgy/yanjie/gossipcop_dataset/gossipcop_v3-1_style_based_fake_test.json")
test_set

,origin_id,generated_text,generated_label
0,gossipcop-7572618844,"On the 132nd day of his presidency, President ...",fake
1,gossipcop-6378885694,"The father of Lady Gaga, known professionally ...",fake
2,gossipcop-909478,The Real Housewives of Atlanta has never been ...,real
3,gossipcop-928269,"In a shocking turn of events, former US Presid...",real
4,gossipcop-924061,"Oh, how my heart aches for her. Baskin Champio...",real
...,...,...,...
2462,gossipcop-850428,"""Mean Girls"" Musical Returns with a Twist: Can...",real
2463,gossipcop-860551,"""Bourne"" actress Julia Stiles and her actor/ca...",real
2464,gossipcop-922801,"""Russell Simmons Accused of Rape in New Lawsui...",real
2465,gossipcop-7704079622,"Gal Gadot, the Israeli actress who gained popu...",fake


In [50]:
# 检查是否能打开？
test_image_path = test_set["origin_id"].map(lambda x: img_dir+"/" + str(x)+"_top_img.png").tolist()
test_image_path

for i in test_image_path:
    try:
        im = Image.open(i, 'r').convert('RGB')
    except Exception as e:
        print(e, i)

cannot identify image file 'top_img/gossipcop-1202546208_top_img.png' top_img/gossipcop-1202546208_top_img.png


In [53]:
test_set[test_set.origin_id == "gossipcop-1202546208"]

,origin_id,generated_text,generated_label
2450,gossipcop-1202546208,"On February 17, 2017, Britain's Daily Mail new...",fake


In [63]:
# 删除这个数据，由于这个图片打不开
afer_drop_test = test_set[test_set.origin_id != "gossipcop-1202546208"]

In [64]:
afer_drop_test.to_json("/home/zgy/yanjie/gossipcop_dataset/gossipcop_v3-1_style_based_fake_test.json", orient="records", indent=4,)